In [1]:
import sys
import os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from logger_config import logger

from langgraph.graph import StateGraph, START, END, add_messages
from typing import TypedDict, Annotated, List
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage


from dotenv import load_dotenv

load_dotenv()

True

In [2]:
memory = MemorySaver()

search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm = ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools = llm.bind_tools(tools=tools)

In [3]:
class BasicState(TypedDict): 
    messages: Annotated[List, add_messages]


In [4]:
def model(state: BasicState): 
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

def tools_router(state: BasicState): 
    last_message = state["messages"][-1]
    if(hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tools"
    else: 
        return END


In [5]:
graph = StateGraph(BasicState)
graph.add_node("model", model)
graph.add_node("tools", ToolNode(tools=tools))

graph.set_entry_point("model")
graph.add_conditional_edges("model", tools_router)

graph.add_edge("tools", "model")

app = graph.compile(checkpointer=memory, interrupt_before=["tools"])

In [8]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

try:
    display(Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API)))
except Exception as e:
    logger.error(f"Unable to draw graph due to error: {str(e)}")

2025-05-16 12:55:26 | 3516219494.py | <module> | Line: 7 | ERROR | Unable to draw graph due to error: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)


In [18]:
config = {"configurable": {
    "thread_id": 1
}}

events = app.stream({
    "messages": [HumanMessage(content="What is the current weather in Hyderabad?")]
}, config=config, stream_mode="values")

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the current weather in Hyderabad?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_7549)
 Call ID: call_7549
  Args:
    query: Hyderabad current weather


In [19]:
snapshot = app.get_state(config=config)
snapshot.next

('tools',)

In [20]:
events = app.stream(None, config, stream_mode="values")
for event in events:
    event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_7549)
 Call ID: call_7549
  Args:
    query: Hyderabad current weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://timesofindia.indiatimes.com/weather/hyderabad-weather-forecast-today/500002", "content": "Today's Weather in Hyderabad: In Hyderabad today, the weather is expected to be Fair with a maximum temperature of 35°C and a minimum of 23°C. Sunrise in"}, {"url": "https://world-weather.info/forecast/india/hyderabad/may-2025/", "content": "Weather in Hyderabad in May 2025. Hyderabad Weather Forecast for May 2025 is ... Thursday, 15 May. +90°. Day. +102°. Clear sky. Friday, 16 May. +86°. Day. +100"}]
================================== Ai Message ==================================

The current weather in Hyderabad is expected to be fair with a maximum temperature of 35°C 